In [0]:
#df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/salesorderdetails-1.csv")
#df2 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_productDescCulture.csv")
#df3 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/salesorderheader.csv")
#df4 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_productDescription.csv")
#df5 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_productmodel.csv")
#df6 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_product.csv")
#df6.show(5)
#df5.show(5)

+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+--------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|       SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+--------------------+-

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("ProductionJoinJob").getOrCreate()

# Load the two tables into DataFrames
# Assuming you are reading from a database or a source, replace with actual reading logic (e.g., JDBC)
product_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_product.csv")

product_model_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_productmodel.csv")

# Rename the 'Name' column in one of the DataFrames to avoid ambiguity
product_df = product_df.withColumnRenamed("Name", "product_name")
#product_model_df = product_model_df.withColumnRenamed("Name", "product_name")

# Sort DataFrames by ProductModelID
sorted_product_df = product_df.orderBy(col("ProductModelID").asc())
sorted_product_model_df = product_model_df.orderBy(col("ProductModelID").asc())

# Select and rename columns for clarity in the final output
sorted_product_df = product_df.select(
    "ProductModelID", 
    "productID", 
    "product_name", 
    "Reorderpoint", 
    "StandardCost",
    "ProductNumber" 
)

sorted_product_model_df = sorted_product_model_df.select(
    "Name",
    "CatalogDescription",
    "ProductModelID"
)

# Perform the left outer join on ProductModelID
joined_df = sorted_product_df.join(sorted_product_model_df, on="ProductModelID", how="left")

# Show the results of the join
joined_df.show(5)

# Stop the Spark session
#spark.stop()


+--------------+---------+--------------------+------------+------------+-------------+----+------------------+
|ProductModelID|productID|        product_name|Reorderpoint|StandardCost|ProductNumber|Name|CatalogDescription|
+--------------+---------+--------------------+------------+------------+-------------+----+------------------+
|          NULL|        1|     Adjustable Race|         750|        0.00|      AR-5381|null|              null|
|          NULL|        2|        Bearing Ball|         750|        0.00|      BA-8327|null|              null|
|          NULL|        3|     BB Ball Bearing|         600|        0.00|      BE-2349|null|              null|
|          NULL|        4|Headset Ball Bear...|         600|        0.00|      BE-2908|null|              null|
|          NULL|      316|               Blade|         600|        0.00|      BL-2036|null|              null|
+--------------+---------+--------------------+------------+------------+-------------+----+------------

In [0]:
product_df.show(5)
product_model_df.show(5)


+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+--------------------+-----------+----------------+--------------------+--------------------+
|ProductID|        product_name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|       SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+--------------------+-

In [0]:
# Ensure the data types are appropriate for sorting and joining
# Convert ProductModelID to integer if necessary (assuming it's stored as string in CSV)
product_df = product_df.withColumn("ProductModelID", col("ProductModelID").cast("int"))
product_model_df = product_model_df.withColumn("ProductModelID", col("ProductModelID").cast("int"))

# Sort DataFrames by ProductModelID in ascending order
sorted_product_df = product_df.orderBy(col("ProductModelID").asc())
sorted_product_model_df = product_model_df.orderBy(col("ProductModelID").asc())


In [0]:


# Load the two tables into DataFrames
# Assuming you are reading from a database or a source, replace with actual reading logic (e.g., JDBC)
product_descr_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_productDescription.csv")
product_modeldescrCult_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_productDescCulture.csv")


# Sort DataFrames by ProductModelID
sorted_product_descr_df = product_descr_df.orderBy(col("ProductDescriptionID").asc())
sorted_product_modeldescrCult_df = product_modeldescrCult_df.orderBy(col("ProductDescriptionID").asc())

# Select and rename columns for clarity in the final output
sorted_product_descr_df = sorted_product_descr_df.select(
    "ProductDescriptionID", 
    "Description"
)

sorted_product_modeldescrCult_df = product_modeldescrCult_df.select(
    "ProductDescriptionID",
    "ProductModelID",
    "CultureID"
)

# Perform the left outer join on ProductModelID
merge_join = sorted_product_descr_df.join(sorted_product_modeldescrCult_df, on="ProductDescriptionID", how="inner")

# Show the results of the join
merge_join.show(5)


# Stop the Spark session
#spark.stop()



+--------------------+--------------------+--------------+---------+
|ProductDescriptionID|         Description|ProductModelID|CultureID|
+--------------------+--------------------+--------------+---------+
|                   3|     Chromoly steel.|            95|   en    |
|                   4|Aluminum alloy cu...|            96|   en    |
|                   5|Aluminum alloy cu...|            97|   en    |
|                   8|Suitable for any ...|            23|   en    |
|                  64|This bike deliver...|            22|   en    |
+--------------------+--------------------+--------------+---------+
only showing top 5 rows



In [0]:
# Sort the DataFrame by ProductModelID in ascending order
sorted_df = merge_join.orderBy("ProductModelID")

# Show the sorted data
sorted_df.show(5)


+--------------------+--------------------+--------------+---------+
|ProductDescriptionID|         Description|ProductModelID|CultureID|
+--------------------+--------------------+--------------+---------+
|                1199|Light-weight, win...|             1|   en    |
|                1467|علب خفيفة الوزن، ...|             1|   ar    |
|                1589|Sacs légers et ré...|             1|   fr    |
|                1712|น้ำหนักเบา กันลม ...|             1|   th    |
|                1838|קל-משקל, מגן מרוח...|             1|   he    |
+--------------------+--------------------+--------------+---------+
only showing top 5 rows



In [0]:
# Select only the relevant columns from each DataFrame for the join
joined_df1 = joined_df.join(merge_join, on="ProductModelID", how="inner")

# Show the results of the join
joined_df1.show(5)

+--------------+---------+--------------------+------------+------------+-------------+-----------------+--------------------+--------------------+--------------------+---------+
|ProductModelID|productID|        product_name|Reorderpoint|StandardCost|ProductNumber|             Name|  CatalogDescription|ProductDescriptionID|         Description|CultureID|
+--------------+---------+--------------------+------------+------------+-------------+-----------------+--------------------+--------------------+--------------------+---------+
|            95|      994|   LL Bottom Bracket|         375|     23.9716|      BB-7421|LL Bottom Bracket|                NULL|                   3|     Chromoly steel.|   en    |
|            96|      995|   ML Bottom Bracket|         375|     44.9506|      BB-8107|ML Bottom Bracket|                NULL|                   4|Aluminum alloy cu...|   en    |
|            97|      996|   HL Bottom Bracket|         375|     53.9416|      BB-9108|HL Bottom Bracket|

In [0]:
from pyspark.sql.functions import upper

# Convert the 'Name' column to uppercase using withColumn
joined_df1_upper = joined_df1.withColumn("Upper_Name", upper(joined_df1["Name"]))

# Show the updated DataFrame
joined_df1_upper.show(5)



+--------------+---------+--------------------+------------+------------+-------------+-----------------+--------------------+--------------------+--------------------+---------+-----------------+
|ProductModelID|productID|        product_name|Reorderpoint|StandardCost|ProductNumber|             Name|  CatalogDescription|ProductDescriptionID|         Description|CultureID|       Upper_Name|
+--------------+---------+--------------------+------------+------------+-------------+-----------------+--------------------+--------------------+--------------------+---------+-----------------+
|            95|      994|   LL Bottom Bracket|         375|     23.9716|      BB-7421|LL Bottom Bracket|                NULL|                   3|     Chromoly steel.|   en    |LL BOTTOM BRACKET|
|            96|      995|   ML Bottom Bracket|         375|     44.9506|      BB-8107|ML Bottom Bracket|                NULL|                   4|Aluminum alloy cu...|   en    |ML BOTTOM BRACKET|
|            97

In [0]:
joined_df1_upper.printSchema()

root
 |-- ProductModelID: string (nullable = true)
 |-- productID: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- Reorderpoint: string (nullable = true)
 |-- StandardCost: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- CatalogDescription: string (nullable = true)
 |-- ProductDescriptionID: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- CultureID: string (nullable = true)
 |-- Upper_Name: string (nullable = true)



In [0]:
# Register the DataFrame as a temporary SQL table
joined_df1_upper.createOrReplaceTempView("product_data")

# SQL query to split the ProductNumber into Category and ProductCarNbr
sql_query = """
SELECT 
    ProductNumber,
    LEFT(ProductNumber, INSTR(ProductNumber, '-') - 1) AS Category,
    CAST(SUBSTRING(ProductNumber, INSTR(ProductNumber, '-') + 1) AS INT) AS ProductCarNbr
FROM product_data
"""

# Execute the SQL query
df_split_sql = spark.sql(sql_query)

# Show the resulting DataFrame
df_split_sql.show(truncate=False)


+-------------+--------+-------------+
|ProductNumber|Category|ProductCarNbr|
+-------------+--------+-------------+
|BB-7421      |BB      |7421         |
|BB-8107      |BB      |8107         |
|BB-9108      |BB      |9108         |
|BK-M18B-52   |BK      |null         |
|BK-M18B-48   |BK      |null         |
|BK-M18B-44   |BK      |null         |
|BK-M18B-42   |BK      |null         |
|BK-M18B-40   |BK      |null         |
|BK-M18S-52   |BK      |null         |
|BK-M18S-48   |BK      |null         |
|BK-M18S-44   |BK      |null         |
|BK-M18S-42   |BK      |null         |
|BK-M18S-40   |BK      |null         |
|BK-M38S-46   |BK      |null         |
|BK-M38S-42   |BK      |null         |
|BK-M38S-40   |BK      |null         |
|BK-M38S-38   |BK      |null         |
|BK-M47B-48   |BK      |null         |
|BK-M47B-44   |BK      |null         |
|BK-M47B-40   |BK      |null         |
+-------------+--------+-------------+
only showing top 20 rows



In [0]:
# Register the DataFrame as a temporary SQL table
joined_df1_upper.createOrReplaceTempView("product_data")

# SQL query to split the ProductNumber into Category and ProductCarNbr
sql_query = """
SELECT 
    *,
    LEFT(ProductNumber, INSTR(ProductNumber, '-') - 1) AS Category,
    CAST(SUBSTRING(ProductNumber, INSTR(ProductNumber, '-') + 1) AS INT) AS ProductCarNbr
FROM product_data
"""

# Execute the SQL query
df_split_sql = spark.sql(sql_query)

# Show the resulting DataFrame
df_split_sql.show(truncate=False)

+--------------+---------+-------------------------+------------+------------+-------------+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-----------------+--------+-------------+
|ProductModelID|productID|product_name             |Reorderpoint|StandardCost|ProductNumber|Name             |CatalogDescription                           

In [0]:
df_split_sql.write.format("csv").mode("overwrite").option("header", "true").save("dbfs:/FileStore/tables/dimension/Data_warehouse_dim_product_sale")

# Show a sample of the final dimension table
df_split_sql.show(5)

# Stop the Spark session
#spark.stop()

+--------------+---------+--------------------+------------+------------+-------------+-------------+------------------+--------------------+-------------------------------------+---------+-------------+--------+-------------+
|ProductModelID|productID|        product_name|Reorderpoint|StandardCost|ProductNumber|         Name|CatalogDescription|ProductDescriptionID|                          Description|CultureID|   Upper_Name|Category|ProductCarNbr|
+--------------+---------+--------------------+------------+------------+-------------+-------------+------------------+--------------------+-------------------------------------+---------+-------------+--------+-------------+
|             6|      680|HL Road Frame - B...|         375|     1059.31|   FR-R92B-58|HL Road Frame|              NULL|                1948|重量最轻、质量最好的铝制车架采用最...|   zh-cht|HL ROAD FRAME|      FR|         null|
|             6|      680|HL Road Frame - B...|         375|     1059.31|   FR-R92B-58|HL Road Frame|        